## Amazon AgentCore Bedrock 코드 인터프리터를 활용한 에이전트 기반 코드 실행 - 튜토리얼(스트랜드)
이 튜토리얼에서는 Python을 사용하여 코드 실행을 통해 답변을 검증하는 AI 에이전트를 만드는 방법을 보여줍니다. LLM에서 생성된 코드를 실행하기 위해 Amazon Bedrock AgentCore Code Interpreter를 사용합니다.

이 튜토리얼에서는 AgentCore Bedrock Code Interpreter를 사용하여 다음 작업을 수행하는 방법을 보여줍니다.
1. 샌드박스 환경 설정
2. 사용자 쿼리를 기반으로 코드를 생성하는 스트랜드 기반 에이전트 구성
3. 코드 인터프리터를 사용하여 샌드박스 환경에서 코드 실행
4. 결과를 사용자에게 표시

## 필수 조건
- Bedrock AgentCore Code Interpreter 액세스 권한이 있는 AWS 계정
- 코드 인터프리터 리소스를 생성하고 관리하는 데 필요한 IAM 권한 보유
- 필요한 Python 패키지 설치(boto3, bedrock-agentcore 및 strands 포함)
- IAM 역할에 Amazon Bedrock에서 모델을 호출할 수 있는 권한이 있어야 함
- 미국 오리건(us-west-2) 리전의 Claude 3.7 Sonnet 모델(Strands SDK의 기본 모델)에 대한 액세스 권한

## IAM 실행 역할에 다음 IAM 정책이 연결되어 있어야 함

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## 작동 방식

코드 실행 샌드박스는 코드 인터프리터, 셸, 파일 시스템을 갖춘 격리된 환경을 생성하여 에이전트가 사용자 쿼리를 안전하게 처리할 수 있도록 합니다. 대규모 언어 모델(LLM)을 통해 도구 선택을 돕고, 코드는 이 세션 내에서 실행된 후 합성을 위해 사용자 또는 에이전트에게 반환됩니다.

![architecture local](code-interpreter.png)

## 1. 환경 설정

먼저, 필요한 라이브러리를 가져오고 코드 인터프리터 세션을 초기화해 보겠습니다.

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import code_session
from strands import Agent, tool
from strands.models import BedrockModel
import json
from typing import Dict, Any, List


## 2. 시스템 프롬프트 정의
AI 비서의 동작과 기능을 정의합니다. AI 비서가 코드 실행 및 데이터 기반 추론을 통해 항상 답변을 검증하도록 지시합니다.

In [ ]:
SYSTEM_PROMPT = """당신은 코드 실행을 통해 모든 답변을 검증하는 유용한 AI 비서입니다.

검증 원칙:
1. 코드, 알고리즘 또는 계산에 대한 주장을 할 때는 이를 검증하는 코드를 작성하세요.
2. execute_python을 사용하여 수학적 계산, 알고리즘 및 논리를 테스트하세요.
3. 답변을 제공하기 전에 이해도를 검증하기 위한 테스트 스크립트를 작성하세요.
4. 항상 실제 코드 실행을 통해 작업을 보여주세요.
5. 확실하지 않은 경우, 제약 사항을 명시적으로 명시하고 검증 가능한 부분을 검증하세요.

접근 방식:
- 프로그래밍 개념에 대한 질문이 있는 경우, 코드로 구현하여 시연하세요.
- 계산을 요청받은 경우, 프로그래밍 방식으로 계산하고 코드를 보여주세요.
- 알고리즘을 구현하는 경우, 정확성을 증명하는 테스트 사례를 포함하세요.
- 투명성을 위해 검증 프로세스를 문서화하세요.
- 샌드박스는 실행 간에 상태를 유지하므로 이전 결과를 참조할 수 있습니다.

사용 가능한 도구:
- execute_python: Python 코드를 실행하고 출력을 확인하세요.

응답 형식: execute_python 도구는 다음을 포함하는 JSON 응답을 반환합니다.
- sessionId: 샌드박스 세션 ID
- id: 요청 ID
- isError: 오류를 나타내는 부울 값 오류가 발생했습니다.
- content: 유형과 텍스트/데이터를 포함하는 콘텐츠 객체 배열
- structuredContent: 코드 실행을 위한 것으로, stdout, stderr, exitCode, executionTime을 포함합니다."""

## 3. 코드 실행 도구 정의
다음으로, 에이전트가 코드 샌드박스에서 코드를 실행하는 데 사용할 함수를 도구로 정의합니다. @tool 데코레이터를 사용하여 해당 함수에 에이전트용 사용자 지정 도구라는 주석을 추가합니다.

활성 코드 인터프리터 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간 상태를 유지할 수 있습니다.

In [ ]:
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""
    
    if description:
        code = f"# {description}\n{code}"
    
    print(f"\n Generated Code: {code}")
    
    with code_session("us-west-2") as code_client:
        response = code_client.invoke("executeCode", {
            "code": code,
            "language": "python",
            "clearContext": False
        })
    
    for event in response["stream"]:
        return json.dumps(event["result"])

## 4. 에이전트 구성
Strands SDK를 사용하여 에이전트를 생성하고 구성합니다. 에이전트에 시스템 프롬프트와 위에서 정의한 도구를 제공하여 코드를 실행합니다.

In [ ]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
)

agent = Agent(
    model=bedrockmodel,
    tools=[execute_python],
    system_prompt=SYSTEM_PROMPT,
    callback_handler=None
)

## 5. 쿼리 정의
에이전트의 코드 실행 기능을 테스트하기 위한 샘플 쿼리를 정의합니다.

In [ ]:
query = "1과 100 사이의 숫자 중 84보다 작고 9보다 큰 가장 큰 난수 소수를 알려주세요."

## 6. 에이전트 호출 및 응답 처리
쿼리를 사용하여 에이전트를 호출하고 에이전트의 응답을 처리합니다.

참고: 비동기 실행은 비동기 환경에서 실행해야 합니다.

In [ ]:
try:
    response_text = ""
    async for event in agent.stream_async(query):
        if "data" in event:
            chunk = event["data"]
            response_text += chunk
            print(chunk, end="")
except Exception as e:
    print(f"Error occurred: {str(e)}")